## Build model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

In [ ]:
def predict_proba(model, texts, tokenizer):
    
    batch = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    
    with torch.no_grad():        
        outputs = model(**batch)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        probs = predictions.detach().numpy().tolist()
        
        return probs

In [ ]:
predict_proba(model, ["good", "bad"], tokenizer)

# UNBOX

In [ ]:
import unboxapi
from unboxapi.models import ModelType
from unboxapi.tasks import TaskType
client = unboxapi.UnboxClient("YOUR_API_KEY_HERE")

# Package & Upload to Unbox

### Upload model

In [ ]:
# Comment this out and uncomment the next section to load the project
project = client.create_project(
    name="Transformer Demo Project",
    description="Project to Demo Transformers with Unbox",
    task_type=TaskType.TextClassification,
)

# Use this for loading the project on subsequent runs
'''
project = client.load_project(
    name="Transformer Demo Project"
)
'''

model = project.add_model(
    function=predict_proba, 
    model=model,
    model_type=ModelType.transformers,
    class_names=['negative', 'positive'],
    name='transformers.sentiment_analyzer',
    description='this is my transformers sentiment model',
    tokenizer=tokenizer,
    requirements_txt_file='./requirements.txt'
)
model.to_dict()